In [ ]:
# HOM_FTP_PATH = r"/home/ftp/ftp/dev/wsr-public/data_output"
# bob.refresh{"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}
# bob.static{"token": ("4622c4cc3822f6c1164cadbd53ed0d"
#                      "1230fb0a562d7c6bc824116cd82a78"),
#            "path": os.path.join(HOM_FTP_PATH, "France_HOSP_DEP_NAME.html"),
#            "filename": "../data_output/France_HOSP_DEP_NAME.html"}
# bob.static{"token": ("24e8978d3f09f857f81ed59d08c2b0"
#                      "1a98e7eb0a95b85d985158b449bf0a"),
#            "path": os.path.join(HOM_FTP_PATH, "France_REA_DEP_NAME.html"),
#            "filename": "../data_output/France_REA_DEP_NAME.html"}
# bob.static{"token": ("83596ba6c97aa043e1994c93e4fc49"
#                      "354cd31842c2f83d66da0ce9dc314f"),
#            "path": os.path.join(HOM_FTP_PATH, "France_DC_DEP_NAME.html"),
#            "filename": "../data_output/France_DC_DEP_NAME.html"}
# bob.static{"token": ("09e1f0c28332e7b570cb13fe66def2"
#                      "187a457697a9a54921c96cfb2f76fd"),
#            "path": os.path.join(HOM_FTP_PATH, "France_RAD_DEP_NAME.html"),
#            "filename": "../data_output/France_RAD_DEP_NAME.html"}

# bob.static{"token": ("113a1888cfa76275418845a0a21fbe"
#                      "d1e2f766bf12a0a7b9b72c20de1aa5"),
#            "path": os.path.join(HOM_FTP_PATH, "France_HOSP_REG_NAME.html"),
#            "filename": "../data_output/France_HOSP_REG_NAME.html"}
# bob.static{"token": ("1bf05f448efec2591a9b59b2a8683b"
#                      "24eb99eaf3314f7a6d87937d05f7f4"),
#            "path": os.path.join(HOM_FTP_PATH, "France_REA_REG_NAME.html"),
#            "filename": "../data_output/France_REA_REG_NAME.html"}
# bob.static{"token": ("a1e16c779ba48901c40ad7daf39dce"
#                      "ea2dda8752b6de257b58108a97b9eb"),
#            "path": os.path.join(HOM_FTP_PATH, "France_DC_REG_NAME.html"),
#            "filename": "../data_output/France_DC_REG_NAME.html"}
# bob.static{"token": ("5fd5e5d8525afa27210b5c64420d47"
#                      "5a531c3b1eb3d6bbd9fed979f3081d"),
#            "path": os.path.join(HOM_FTP_PATH, "France_RAD_REG_NAME.html"),
#            "filename": "../data_output/France_RAD_REG_NAME.html"}

# bob.static{"token": ("3a0e8e24ed4c4a9631007d76a988bd"
#                      "40660844e2e6ce6a631a528c22774d"),
#            "path": os.path.join(HOM_FTP_PATH, "France_TSHOSP_DEP_NAME.html"),
#            "filename": "../data_output/France_TSHOSP_DEP_NAME.html"}
# bob.static{"token": ("ec820131eb62001106cb3f0a74f2ad"
#                      "bacefb68f5d11854be593972e4f55f"),
#            "path": os.path.join(HOM_FTP_PATH, "France_TSHOSP_REG_NAME.html"),
#            "filename": "../data_output/France_TSHOSP_REG_NAME.html"}

## Database
*italic* <br>
**gras**
* Point 1
* Point 2

In [ ]:
%run __init__.ipynb

In [ ]:
def analysis(dataf, variable):
    """
    doc string
    """
    v_min = dataf[variable].min()
    v_max = dataf[variable].max()
    v_avr = round(dataf[variable].mean(), 0)
    v_med = round(np.median(dataf[variable]), 0)
    avrmax = round((v_avr / v_max), 2)
    medmax = round((v_med / v_max), 2)
    to_print = (f'{variable} : '
                f'min: {v_min}, max: {v_max}, average: {v_avr}, '
                f'avrmax: {avrmax}, med: {v_med}, medmax: {medmax}')
    return print(to_print)

In [ ]:
# Var France
MAP_CENTER = {"lat": 46.7667, "lon": 2.45}


def create_map(dataf, geo, dfkey, geokey, scale,
               colors, map_center, label, file_html):
    """
    doc string
    """
    fig = px.choropleth_mapbox(dataf,
                               geojson=geo,
                               color="VALUE",
                               locations=dfkey,
                               featureidkey=geokey,
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron",
                               zoom=5,
                               center=map_center,
                               hover_data=["VALUE", "VAR"],
                               labels=label)
    fig.update_layout(margin={"r": 0, "l": 0}, autosize=True)
    fig.write_html(f"{OUTPUT_FOLDER}{file_html}")
    updateChartCss(f"{OUTPUT_FOLDER}{file_html}", f"{INPUT_FOLDER}map.css")
    # return fig

In [ ]:
# --- Check if the data are already updated
FILEPATH = f'{OUTPUT_FOLDER}107_FRANCE.csv'
if os.path.exists(FILEPATH):
    DF_CHECK = pd.read_csv(FILEPATH, sep=';', low_memory=False)
    DATE_CHECK = DF_CHECK[DF_CHECK['SCENARIO'] == (YESTERDAY.
                                                   strftime('%d/%m/%Y'))]
    if DATE_CHECK:
        print(f'Data already updated ! DateTime : {now}')
        %stop

In [12]:
# Source 107
DATAF = pd.read_csv(os.path.join(OUTPUT_FOLDER,
                                 '107.csv'), sep=";").drop('Unnamed: 0',
                                                           axis=1)
DATAF = (DATAF[DATAF['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y')].
         reset_index(drop=True))
if not DATAF:
    print(f'Data not updated ! DateTime : {now}')
    %stop
else:
    DATAF.to_csv(os.path.join(OUTPUT_FOLDER, '07_FRANCE.csv'), sep=";")

DATAF = (pd.merge(DATAF, REF_FRANCE,
                  left_on='CHILD', right_on='DEP_FULL', how='left').
         rename(index=str, columns={"DEP_NAME": "LABEL"}).
         drop(['DEP_CODE', 'DEP_FULL', 'REG_NAME'], axis=1))
DATAF.loc[DATAF['PARENT'] == 'root', 'LABEL'] = DATAF['CHILD']
DATAF = DATAF.drop(['UNIT_VARP', 'PRECISION_VALUE', 'PRECISION_VAR',
                    'PRECISION_VARP', 'LAST_UPDATE'], axis=1)
DATAF.head()

,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,LABEL
0,France,03/05/2020,20200503,Île-de-France,root,En réanimation,2.0,1550,-2,-0.128866,Île-de-France
1,France,03/05/2020,20200503,Île-de-France,root,Hospitalisations,1.0,10685,-30,-0.279981,Île-de-France
2,France,03/05/2020,20200503,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,125,3,2.459016,Haute-Garonne
3,France,03/05/2020,20200503,32 - Gers,Occitanie,Hospitalisations,1.0,28,1,3.703704,Gers
4,France,03/05/2020,20200503,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,212,5,2.415459,Gironde


In [13]:
# Hospitalisation - Département
KPI = 'Hospitalisations'
FILE_HTML = "France_HOSP_DEP_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] != 'root')]
analysis(DF_MAP, 'VALUE')

SCALE = (0, 3000)
COLORS = [[0.0, "#FFFFFF"],
          [0.05, "#B3D7D2"],
          [0.10, "#81B1BD"],
          [0.55, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Département',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_DEP, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 2, Max: 2177, Average: 255.0, AvrMax: 0.12, Med: 106.0, MedMax: 0.05


In [14]:
# En réanimation - Département
KPI = 'En réanimation'
FILE_HTML = "France_REA_DEP_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] != 'root')]
analysis(DF_MAP, 'VALUE')

SCALE = (0, 800)
COLORS = [[0.0, "#FFFFFF"],
          [0.025, "#B3D7D2"],
          [0.10, "#81B1BD"],
          [0.55, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Département',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_DEP, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")

VALUE : Min: 0, Max: 430, Average: 37.0, AvrMax: 0.09, Med: 13.0, MedMax: 0.03


In [15]:
# "Décès à l’hôpital" - Département
KPI = "Décès à l’hôpital"
FILE_HTML = "France_DC_DEP_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] != 'root')]
analysis(DF_MAP, 'VALUE')

SCALE = (0, 1500)
COLORS = [[0.0, "#FFFFFF"],
          [0.05, "#B3D7D2"],
          [0.10, "#81B1BD"],
          [0.55, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Département',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_DEP, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")

VALUE : Min: 0, Max: 1493, Average: 154.0, AvrMax: 0.1, Med: 60.0, MedMax: 0.04


In [16]:
# Retours à domicile' - Département
KPI = 'Retours à domicile'
FILE_HTML = "France_RAD_DEP_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] != 'root')]
analysis(DF_MAP, 'VALUE')

SCALE = (0, 3000)
COLORS = [[0.0, "#FFFFFF"],
          [0.05, "#B3D7D2"],
          [0.10, "#81B1BD"],
          [0.55, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Département',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_DEP, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")

VALUE : Min: 18, Max: 4229, Average: 503.0, AvrMax: 0.12, Med: 178.0, MedMax: 0.04


# ANALYSIS BY REGION

In [17]:
# Hospitalisation - Région
KPI = 'Hospitalisations'
FILE_HTML = "France_HOSP_REG_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] == 'root') &
                      (DATAF['LABEL'] != 'France')]
analysis(DF_MAP, 'VALUE')

SCALE = (100, 13000)
COLORS = [[0.0, "#FFFFFF"],
          [0.075, "#B3D7D2"],
          [0.15, "#81B1BD"],
          [0.4, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Région',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_REG, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")

VALUE : Min: 2, Max: 10685, Average: 1430.0, AvrMax: 0.13, Med: 601.0, MedMax: 0.06


In [18]:
# En réanimation - Région
KPI = 'En réanimation'
FILE_HTML = "France_REA_REG_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] == 'root') &
                      (DATAF['LABEL'] != 'France')]
analysis(DF_MAP, 'VALUE')

SCALE = (50, 2500)
COLORS = [[0.0, "#FFFFFF"],
          [0.075, "#B3D7D2"],
          [0.15, "#81B1BD"],
          [0.55, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Région',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_REG, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")

VALUE : Min: 0, Max: 1550, Average: 209.0, AvrMax: 0.13, Med: 98.0, MedMax: 0.06


In [19]:
# "Décès à l’hôpital" - Région
KPI = "Décès à l’hôpital"
FILE_HTML = "France_DC_REG_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] == 'root') &
                      (DATAF['LABEL'] != 'France')]
analysis(DF_MAP, 'VALUE')

SCALE = (100, 5500)
COLORS = [[0.0, "#FFFFFF"],
          [0.075, "#B3D7D2"],
          [0.15, "#81B1BD"],
          [0.4, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Région',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_REG, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")

VALUE : Min: 0, Max: 6116, Average: 865.0, AvrMax: 0.14, Med: 351.0, MedMax: 0.06


In [20]:
# Retours à domicile - Région
KPI = 'Retours à domicile'
FILE_HTML = "France_RAD_REG_NAME.html"

DF_MAP = DATAF.copy()[(DATAF['UPPER_FILTER_M'] == KPI) &
                      (DATAF['PARENT'] == 'root') &
                      (DATAF['LABEL'] != 'France')]
analysis(DF_MAP, 'VALUE')

SCALE = (200, 15000)
COLORS = [[0.0, "#FFFFFF"],
          [0.075, "#B3D7D2"],
          [0.15, "#81B1BD"],
          [0.4, "#437F9D"],
          [1.0, "#00354E"]]
DFKEY = "LABEL"
GEOKEY = "properties.nom"
TO_RENAME = {'LABEL': 'Région',
             'VALUE': KPI,
             'VAR': 'Variation en nb.'}
create_map(DF_MAP, FRANCE_REG, DFKEY, GEOKEY, SCALE,
           COLORS, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")

VALUE : Min: 36, Max: 18056, Average: 2821.0, AvrMax: 0.16, Med: 1262.0, MedMax: 0.07


In [21]:
MAP_CENTER = {"lat": 46.7667, "lon": 2.45}


def create_map_epid(dataf, geo, dfkey, geokey, col_color,
                    hoverdata, map_center, label, file_html):
    """
    doc string

    replaces default color mapping by value
    """
    fig = px.choropleth_mapbox(dataf,
                               geojson=geo,
                               color=col_color,
                               locations=dfkey,
                               featureidkey=geokey,
                               color_discrete_map={"vert": "#4CCD88",
                                                   "orange": "#FDB47A",
                                                   "rouge": "#DD6D88"},
                               mapbox_style="carto-positron",
                               zoom=5,
                               center=map_center,
                               hover_data=hoverdata,
                               labels=label,
                               opacity=0.8)
    fig.update_layout(margin={"r": 0, "l": 0}, autosize=True, showlegend=False)
    fig.write_html(f"{OUTPUT_FOLDER}{file_html}")
    update_chart_css(f"{OUTPUT_FOLDER}{file_html}", f"{INPUT_FOLDER}map.css")
    return fig

In [22]:
# --- Situation epidémiologique DATA
COLS = ['DEP', 'DEP_NAME', 'REA_2013', 'REA_2018', 'SI_2013',
        'SI_2018', 'SC_2013', 'SC_2018', 'ALL_2013', 'ALL_2018']
FICHIER = 'drees_lits_reanimation_2013-2018.xlsx'
REF_FRANCE_LITS = pd.read_excel(os.path.join(INPUT_FOLDER, FICHIER),
                                sheet_name='Lits_2013-2018',
                                header=7, names=COLS)
REF_FRANCE_LITS = REF_FRANCE_LITS[['DEP', 'DEP_NAME', 'REA_2018']]
REF_FRANCE_LITS['DEP'] = REF_FRANCE_LITS['DEP'].astype(str)
REF_FRANCE_LITS.head()

,DEP,DEP_NAME,REA_2018
0,FR,France entière,5432
1,MET,France métropolitaine,5290
2,01,Ain,22
3,02,Aisne,34
4,03,Allier,24


In [23]:
# ---Tension Data
DF_REA = pd.read_csv(os.path.join(OUTPUT_FOLDER, '107_FRANCE.csv'),
                     sep=";").drop('Unnamed: 0', axis=1)
DF_REA = DF_REA[['ENTITY', 'SCENARIO', 'CHILD',
                 'PARENT', 'UPPER_FILTER_M', 'VALUE', 'VAR']]
DF_REA = (pd.merge(DF_REA, REF_FRANCE,
                   left_on='CHILD', right_on='DEP_FULL', how='left').
          rename(index=str, columns={"DEP_NAME": "LABEL"}).
          drop(['DEP_FULL'], axis=1))
DF_REA.loc[DF_REA['PARENT'] == 'root', 'LABEL'] = DF_REA['CHILD']
DF_REA['DEP_CODE'] = DF_REA['DEP_CODE'].astype(str)

DF_REA.loc[DF_REA['CHILD'] == 'France', 'DEP_CODE'] = 'FR'
DF_REA = (pd.merge(DF_REA, REF_FRANCE_LITS,
                   left_on='DEP_CODE', right_on='DEP', how='left').
          drop(['DEP', 'DEP_NAME'], axis=1))
DF_REA.head()

,ENTITY,SCENARIO,CHILD,PARENT,UPPER_FILTER_M,VALUE,VAR,DEP_CODE,LABEL,REG_NAME,REA_2018
0,France,03/05/2020,Île-de-France,root,En réanimation,1550,-2,nan,Île-de-France,NaN,NaN
1,France,03/05/2020,Île-de-France,root,Hospitalisations,10685,-30,nan,Île-de-France,NaN,NaN
2,France,03/05/2020,31 - Haute-Garonne,Occitanie,Hospitalisations,125,3,31,Haute-Garonne,Occitanie,165.0
3,France,03/05/2020,32 - Gers,Occitanie,Hospitalisations,28,1,32,Gers,Occitanie,8.0
4,France,03/05/2020,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,212,5,33,Gironde,Nouvelle-Aquitaine,195.0


In [24]:
# ---HKPIS
DF_HKPIS = DF_REA.copy()[DF_REA['CHILD'] == 'France']
DF_KPIS = DF_HKPIS[['ENTITY', 'SCENARIO', 'UPPER_FILTER_M', 'VALUE', 'VAR']]

DF_REAKPI = (DF_HKPIS.copy()
             [DF_HKPIS['UPPER_FILTER_M'] == 'En réanimation']
             [['ENTITY', 'SCENARIO', 'UPPER_FILTER_M',
               'VALUE', 'VAR', 'REA_2018']])
DF_REAKPI['VALUEP'] = round((DF_REAKPI['VALUE'] /
                             DF_REAKPI['REA_2018'] * 100), 2)
DF_REAKPI['VALUE_N1'] = (DF_REAKPI['VALUE'] - DF_REAKPI['VAR'])
DF_REAKPI['VALUEP_N1'] = round(((DF_REAKPI['VALUE_N1'] /
                                 DF_REAKPI['REA_2018']) * 100), 2)
DF_REAKPI['VALUEP_VAR'] = DF_REAKPI['VALUEP'] - DF_REAKPI['VALUEP_N1']

DF_REAKPI = (DF_REAKPI.
             drop(['VALUE', 'VAR', 'REA_2018',
                   'VALUE_N1', 'VALUEP_N1'], axis=1).
             rename(index=str, columns={"VALUEP": "VALUE",
                                        "VALUEP_VAR": "VAR"}))
DF_REAKPI['UPPER_FILTER_M'] = 'Tensions hospitalières'

DF_FINAL = pd.concat([DF_KPIS, DF_REAKPI], axis=0)
DF_FINAL.loc[DF_FINAL['UPPER_FILTER_M'] == 'Tensions hospitalières',
             'UNIT_VALUE'] = ' %'
DF_FINAL.loc[DF_FINAL['UPPER_FILTER_M'] == 'Tensions hospitalières',
             'UNIT_VAR'] = ' pts'
DF_FINAL.loc[:, 'PRECISION'] = ',.0f'
DF_FINAL.loc[DF_FINAL['UPPER_FILTER_M'] == 'Tensions hospitalières',
             'PRECISION'] = ',.2f'

if USE_MONGO:
    bob.mongo.save_df(DF_FINAL, '110_BIS', DB_APP, True)
DF_FINAL

Dataframe 110_BIS successfully save in database app-wsr-dev in MongoDB. Time: --- 0.018590927124023438 secnds ---


,ENTITY,SCENARIO,UPPER_FILTER_M,VALUE,VAR,UNIT_VALUE,UNIT_VAR,PRECISION
77,France,03/05/2020,Hospitalisations,25739.00,-12.00,NaN,NaN,",.0f"
199,France,03/05/2020,Décès à l’hôpital,15564.00,96.00,NaN,NaN,",.0f"
316,France,03/05/2020,En réanimation,3762.00,-8.00,NaN,NaN,",.0f"
436,France,03/05/2020,Retours à domicile,50776.00,222.00,NaN,NaN,",.0f"
316,France,03/05/2020,Tensions hospitalières,69.26,-0.14,%,pts,",.2f"


In [25]:
# Par département
DF_REA_DEP = DF_REA.copy()[(DF_REA['UPPER_FILTER_M'] == 'En réanimation') &
                           (DF_REA['PARENT'] != 'root') &
                           (DF_REA['SCENARIO'] == (YESTERDAY.
                                                   strftime('%d/%m/%Y')))]
DF_REA_DEP['VALUEP'] = round((DF_REA_DEP['VALUE'] /
                              DF_REA_DEP['REA_2018'] * 100), 2)
DF_REA_DEP.loc[DF_REA_DEP['VALUEP'] < 60, 'COLOR'] = 'vert'
DF_REA_DEP.loc[DF_REA_DEP['VALUEP'] >= 60, 'COLOR'] = 'orange'
DF_REA_DEP.loc[DF_REA_DEP['VALUEP'] >= 80, 'COLOR'] = 'rouge'
DF_REA_DEP.head()

,ENTITY,SCENARIO,CHILD,PARENT,UPPER_FILTER_M,VALUE,VAR,DEP_CODE,LABEL,REG_NAME,REA_2018,VALUEP,COLOR
242,France,03/05/2020,01 - Ain,Auvergne-Rhône-Alpes,En réanimation,14,0,01,Ain,Auvergne-Rhône-Alpes,22.0,63.64,orange
243,France,03/05/2020,32 - Gers,Occitanie,En réanimation,4,0,32,Gers,Occitanie,8.0,50.00,vert
244,France,03/05/2020,33 - Gironde,Nouvelle-Aquitaine,En réanimation,45,0,33,Gironde,Nouvelle-Aquitaine,195.0,23.08,vert
245,France,03/05/2020,34 - Hérault,Occitanie,En réanimation,29,-1,34,Hérault,Occitanie,128.0,22.66,vert
246,France,03/05/2020,35 - Ille-et-Vilaine,Bretagne,En réanimation,14,0,35,Ille-et-Vilaine,Bretagne,66.0,21.21,vert


In [26]:
# ---Tension_Département
KPI = 'Tension hospitalière (%)'
FILE_HTML = "France_TSHOSP_DEP_NAME.html"
DF_KEY = "LABEL"
GEOKEY = "properties.nom"
COL_COLOR = 'COLOR'
HOVERDATA = ["VALUE", "REA_2018", 'VALUEP']
TO_RENAME = {'COLOR': "Niveau",
             'LABEL': 'Département',
             'VALUE': 'Nombre de cas en réanimation',
             'REA_2018': 'Nombre de lits en service réanimation',
             'VALUEP': KPI}
create_map_epid(DF_REA_DEP, FRANCE_DEP, DFKEY, GEOKEY, COL_COLOR,
                HOVERDATA, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")
# URL = ('https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/'
#        '3a0e8e24ed4c4a9631007d76a988bd40660844e2e6ce6a631a528c22774d')

In [27]:
# Par région
DF_REA_RE = DF_REA.copy()[(DF_REA['UPPER_FILTER_M'] == 'En réanimation') &
                          (DF_REA['PARENT'] != 'root') &
                          (DF_REA['SCENARIO'] == (YESTERDAY.
                                                  strftime('%d/%m/%Y')))]
COLS_TO_GROUP = ['ENTITY', 'SCENARIO', 'REG_NAME']
DF_REA_RE['VALUE'] = DF_REA_RE['VALUE'].astype(int)
DF_REA_RE['VAR'] = DF_REA_RE['VAR'].astype(int)
DF_REA_RE = DF_REA_RE.groupby(COLS_TO_GROUP,
                              as_index=False).agg({"VALUE": 'sum',
                                                   "VAR": 'sum',
                                                   "REA_2018": 'sum'})

DF_REA_RE['VALUEP'] = round((DF_REA_RE['VALUE'] /
                             DF_REA_RE['REA_2018'] * 100), 2)
DF_REA_RE.loc[DF_REA_RE['VALUEP'] < 60, 'COLOR'] = 'vert'
DF_REA_RE.loc[DF_REA_RE['VALUEP'] >= 60, 'COLOR'] = 'orange'
DF_REA_RE.loc[DF_REA_RE['VALUEP'] >= 80, 'COLOR'] = 'rouge'
DF_REA_RE.head()

,ENTITY,SCENARIO,REG_NAME,VALUE,VAR,REA_2018,VALUEP,COLOR
0,France,03/05/2020,Auvergne-Rhône-Alpes,358,-1,609.0,58.78,vert
1,France,03/05/2020,Bourgogne-Franche-Comté,174,2,207.0,84.06,rouge
2,France,03/05/2020,Bretagne,45,0,171.0,26.32,vert
3,France,03/05/2020,Centre-Val de Loire,115,3,191.0,60.21,orange
4,France,03/05/2020,Corse,8,0,18.0,44.44,vert


In [28]:
# ---Tension_Région
KPI = 'Tension hospitalière (%)'
FILE_HTML = "France_TSHOSP_REG_NAME.html"
DFKEY = "REG_NAME"
GEOKEY = "properties.nom"
COL_COLOR = 'COLOR'
HOVERDATA = ["VALUE", "REA_2018", 'VALUEP']
TO_RENAME = {'REG_NAME': 'Région', 'COLOR': "Niveau",
             'VALUE': 'Nombre de cas en réanimation',
             'REA_2018': 'Nombre de lits en service réanimation',
             'VALUEP': KPI}
create_map_epid(DF_REA_RE, FRANCE_REG, DFKEY, GEOKEY, COL_COLOR,
                HOVERDATA, MAP_CENTER, TO_RENAME, FILE_HTML)
# bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER, FILE_HTML), "dev")
# URL = ('https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/'
#        'ec820131eb62001106cb3f0a74f2adbacefb68f5d11854be593972e4f55f')

In [29]:
# html
# Par département
RACINE = 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/'
HOSP_DEP = [{'URL': (RACINE +
                     ('4622c4cc3822f6c1164cadbd53ed'
                      '0d1230fb0a562d7c6bc824116cd82')),
             'KPI': 'Hospitalisations',
             'UPPER_FILTER': 'Par département'}]
REA_DEP = [{'URL': (RACINE +
                    ('24e8978d3f09f857f81ed59d08c2'
                     'b01a98e7eb0a95b85d985158b449bf0a')),
            'KPI': 'En réanimation',
            'UPPER_FILTER': 'Par département'}]
DC_DEP = [{'URL': (RACINE +
                   ('09e1f0c28332e7b570cb13fe66de'
                    'f2187a457697a9a54921c96cfb2f76fd')),
           'KPI': 'Retours à domicile',
           'UPPER_FILTER': 'Par département'}]
RAD_DEP = [{'URL': (RACINE +
                    ('83596ba6c97aa043e1994c93e4fc'
                     '49354cd31842c2f83d66da0ce9dc314f')),
            'KPI': 'Décès à l’hôpital',
            'UPPER_FILTER': 'Par département'}]
REAT_DEP = [{'URL': (RACINE +
                     ('3a0e8e24ed4c4a9631007d76a988'
                      'bd40660844e2e6ce6a631a528c22774d')),
             'KPI': 'Tensions hospitalières',
             'UPPER_FILTER': 'Par département'}]

# Par région
HOSP_REG = [{'URL': (RACINE +
                     ('113a1888cfa76275418845a0a21f'
                      'bed1e2f766bf12a0a7b9b72c20de1aa5')),
             'KPI': 'Hospitalisations',
             'UPPER_FILTER': 'Par région'}]
REA_REG = [{'URL': (RACINE +
                    ('1bf05f448efec2591a9b59b2a868'
                     '3b24eb99eaf3314f7a6d87937d05f7f4')),
            'KPI': 'En réanimation',
            'UPPER_FILTER': 'Par région'}]
DC_REG = [{'URL': (RACINE +
                   ('5fd5e5d8525afa27210b5c64420d'
                    '475a531c3b1eb3d6bbd9fed979f3081d')),
           'KPI': 'Retours à domicile',
           'UPPER_FILTER': 'Par région'}]
RAD_REG = [{'URL': (RACINE +
                    ('a1e16c779ba48901c40ad7daf39d'
                     'ceea2dda8752b6de257b58108a97b9eb')),
            'KPI': 'Décès à l’hôpital',
            'UPPER_FILTER': 'Par région'}]
REAT_REG = [{'URL': (RACINE +
                     ('ec820131eb62001106cb3f0a74f2'
                      'adbacefb68f5d11854be593972e4f55f')),
             'KPI': 'Tensions hospitalières',
             'UPPER_FILTER': 'Par région'}]
        
LIST_KPIS = (HOSP_DEP + REA_DEP + DC_DEP + RAD_DEP +
             HOSP_REG + REA_REG + DC_REG + RAD_REG + REAT_DEP + REAT_REG)
DOMAIN = pd.DataFrame(LIST_KPIS)
DOMAIN.loc[:, 'VALUE'] = ("<iframe src='" +
                          DOMAIN['URL'] +
                          ("' frameborder='0' style='overflow:hidden;"
                           "height:85vh;width:100%' height='100%' "
                           "width='100%'></iframe> <style>.modebar"
                           "{display: none!important;}</style>"))
if USE_MONGO:
    bob.mongo.save_df(DOMAIN, '110', DB_APP, True)
DOMAIN

Dataframe 110 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.01493692398071289 secnds ---


,URL,KPI,UPPER_FILTER,VALUE
0,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par département,<iframe src='https://api.dev.cashstory.com/api...
1,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par département,<iframe src='https://api.dev.cashstory.com/api...
2,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par département,<iframe src='https://api.dev.cashstory.com/api...
3,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par département,<iframe src='https://api.dev.cashstory.com/api...
4,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par région,<iframe src='https://api.dev.cashstory.com/api...
5,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par région,<iframe src='https://api.dev.cashstory.com/api...
6,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par région,<iframe src='https://api.dev.cashstory.com/api...
7,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par région,<iframe src='https://api.dev.cashstory.com/api...
8,https://api.dev.cashstory.com/api/v1/notebook/...,Tensions hospitalières,Par département,<iframe src='https://api.dev.cashstory.com/api...
9,https://api.dev.cashstory.com/api/v1/notebook/...,Tensions hospitalières,Par région,<iframe src='https://api.dev.cashstory.com/api...
